In [1]:
from scipy.io import wavfile
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift, fft, ifft
import soundfile as sf
from scipy.signal import hann
from CircularBuffer import CircularBuffer, StereoDelay


#KYW inst
fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final_Project/Music-Technology-Final-Project/Sound Examples/KYW Inst.wav')
#ATF Drums
#fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final_Project/Music-Technology-Final-Project/Sound Examples/Around the fur drums.wav')

# Get the bit depth of the audio data (assumes audio_data is a NumPy array)
bit_depth = data.dtype.itemsize * 8

# Calculate the scaling factor for normalization
scaling_factor = 2 ** (bit_depth - 1)  # For signed audio

# Convert audio data to floating-point values and normalize
data = data.astype(np.float32) / scaling_factor

# Verify that the data is now in the range -1 to 1
print(f"Min Value: {np.min(data)}")
print(f"Max Value: {np.max(data)}")

Min Value: -0.9484677314758301
Max Value: 0.9998999834060669


## Moorers Reverb
Similar to Schroeders Reverb, except Comb filters used are replaced with LPF comb filters and only one APF is used

In [3]:
# Setting up IIR Lowpass filter
# Can probably be replaced with multiple with different cutoffs for better effect, but just 
# Using one for now to test
fc = 9000
b, a = signal.iirfilter(3, fc, btype='lowpass', ftype='butter', output='ba', fs=fs)

In [12]:
#Comb filters
N = 6
parallel_delayLines = [None] * N
gains = [0.46, 0.47, 0.475, 0.48, 0.49, 0.5]
delays = [0.050, 0.056, 0.061, 0.068, 0.072, 0.078]
output_comb = np.zeros((len(data), 2))

# APF 1
delayTime1 = 0.006
delaySamples1 = delayTime1 * fs
g1 = 0.7
delayLine1 = StereoDelay(delaySamples1, delaySamples1)
output_apf1 = np.zeros((len(data), 2))

# initialise delay lines
for i in range(N):
    parallel_delayLines[i] = StereoDelay((delays[i] * fs), (delays[i] * fs))



for j in range(N):
    for i in range(len(data)):
        #Comb Filter
        delayed = 0

        delayed = parallel_delayLines[j].get_samples_from_buffer()

        output_comb[i] += delayed

        delayed = signal.lfilter(b, a, delayed)

        delayed = np.multiply(delayed, gains[j])

        parallel_delayLines[j].push_samples_to_buffer(delayed + data[i])


for i in range(len(data)):
    #APF1
    delayed1 = delayLine1.get_samples_from_buffer()

    delayLine1.push_samples_to_buffer(output_comb[i] + np.multiply(delayed1, g1))

    output_apf1[i] = (-g1 * output_comb[i]) + np.multiply((1 - pow(g1,2)), delayed1)




In [13]:
#Wet/Dry
wetAmt = 0.5

output_apf1 = np.multiply((1-wetAmt), data) + np.multiply((wetAmt), output_apf1)

In [14]:
#Normalisation
max_amplitude = np.max(np.abs(output_apf1))
normalized_audio = output_apf1 / max_amplitude

In [15]:
sd.play(normalized_audio, fs)